##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#### Some explanation for code

# Since training CNN needs a lot of computation. It is too slow to run locally.
# I run the code on colab. This notebook is directly downloaded from colab.
# It may not be able to run locally.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip '/content/drive/MyDrive/dataset/computer_vision.zip' -d '/content/drive/MyDrive/dataset/computer_vision/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/dataset/computer_vision.zip
   creating: /content/drive/MyDrive/dataset/computer_vision/computer_vision/
   creating: /content/drive/MyDrive/dataset/computer_vision/computer_vision/.ipynb_checkpoints/
  inflating: /content/drive/MyDrive/dataset/computer_vision/computer_vision/.ipynb_checkpoints/GradProject_NB1-checkpoint.ipynb  
  inflating: /content/drive/MyDrive/dataset/computer_vision/computer_vision/.ipynb_checkpoints/GradProject_NB2-checkpoint.ipynb  
  inflating: /content/drive/MyDrive/dataset/computer_vision/computer_vision/.ipynb_checkpoints/GradProject_NB3-checkpoint.ipynb  
  inflating: /content/drive/MyDrive/dataset/computer_vision/computer_vision/.ipynb_checkpoints/Untitled-checkpoint.ipynb  
   creating: /content/drive/MyDrive/dataset/computer_vision/computer_vision/20_categories_training/
   creating: /content/dr

In [49]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import skimage
from skimage import data
from skimage import io
import os
import matplotlib.pyplot as plt
from numpy.linalg import matrix_rank
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import skimage
from skimage import data
from skimage import io
import os
from PIL import Image
import matplotlib.colors as colors
from skimage.feature import hog
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from numpy.linalg import matrix_rank
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import skimage
from skimage import data
from skimage import io
import os
from PIL import Image
import matplotlib.colors as colors
from skimage.feature import hog
import random
from sklearn.preprocessing import StandardScaler
import re

In [50]:
def extract_int(name):
  def convert_int(s):
    try:
      return int(s)
    except:
      return None
  return [convert_int(c) for c in re.split('([0-9]+)', name)]

In [51]:
# Due to dead kernel problem, I cannot use import to do this, so I can only just copy all the code of part 1
def read_organize_data(file_path):
    #Fill this function out, should return a dataframe with picture object, and correct encoding
    data_frame = pd.DataFrame()
    pic_list = []
    encode_list = []
    filenames = os.listdir(file_path)
    filenames.sort(key=extract_int)
    for filename in filenames:
        if filename[0] != '.': # Just readin useful jpg
            name = filename.split('_')[0] #get the picture type
            encode = encoding_dist[name] #get the encode
            file = os.path.join(file_path, filename)
            pic = io.imread(file)
            if len(pic.shape) == 3:
                pic_list.append(pic)
                encode_list.append(encode)
            else:
                img = io.imread(file)
                print(file)
                img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                pic_list.append(img_rgb)
                encode_list.append(encode)
    data_frame['Pictures'] = pic_list
    data_frame['Encoding'] = encode_list
    return data_frame

encoding_dist = {'validation':-1, 'airplanes':0, 'bear':1, 'blimp':2, 'comet':3, 'crab':4, 'dog':5, 'dolphin':6, 'giraffe':7, 'goat':8, 'gorilla':9, 'kangaroo':10\
                , 'killer-whale':11, 'leopards':12, 'llama':13, 'penguin':14, 'porcupine':15, 'teddy-bear':16, 'triceratops':17, 'unicorn':18, 'zebra':19}
# For test set, the label is unknown, so here encode them to -1

def get_train_data(train_path):
    train_data = pd.DataFrame(columns = ['Pictures','Encoding'])
    for types in os.listdir(train_path): #run read_organize_data for each folder(each type of training data)
        if (types[0] >= 'a' and types[0] <= 'z'): 
            filepath = os.path.join(train_path, types)
            train_data_type = read_organize_data(filepath)
            train_data = pd.concat([train_data,train_data_type])
    return train_data

train_path = '/content/drive/MyDrive/dataset/computer_vision/computer_vision/20_categories_training/20_categories_training'
train_data = get_train_data(train_path)
test_data = read_organize_data('/content/drive/MyDrive/dataset/computer_vision/computer_vision/20_Validation')

/content/drive/MyDrive/dataset/computer_vision/computer_vision/20_categories_training/20_categories_training/blimp/blimp_0022.jpg
/content/drive/MyDrive/dataset/computer_vision/computer_vision/20_categories_training/20_categories_training/comet/comet_0006.jpg
/content/drive/MyDrive/dataset/computer_vision/computer_vision/20_categories_training/20_categories_training/comet/comet_0011.jpg
/content/drive/MyDrive/dataset/computer_vision/computer_vision/20_categories_training/20_categories_training/comet/comet_0013.jpg
/content/drive/MyDrive/dataset/computer_vision/computer_vision/20_categories_training/20_categories_training/comet/comet_0021.jpg
/content/drive/MyDrive/dataset/computer_vision/computer_vision/20_categories_training/20_categories_training/comet/comet_0036.jpg
/content/drive/MyDrive/dataset/computer_vision/computer_vision/20_categories_training/20_categories_training/comet/comet_0038.jpg
/content/drive/MyDrive/dataset/computer_vision/computer_vision/20_categories_training/20_c

In [54]:
train_data.to_pickle('train_data.pkl')
train_features = train_data
train_features

,Pictures,Encoding
0,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
1,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
2,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
3,"[[[174, 190, 187], [173, 189, 186], [172, 188,...",0
4,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
...,...,...
59,"[[[159, 129, 105], [155, 125, 99], [152, 122, ...",19
60,"[[[162, 145, 115], [164, 145, 115], [165, 146,...",19
61,"[[[126, 155, 247], [126, 155, 247], [128, 157,...",19
62,"[[[10, 195, 0], [48, 255, 33], [21, 236, 44], ...",19


In [53]:
test_data.to_pickle('test_data.pkl')
test_features = test_data
test_features['Encoding'] = np.load('/content/drive/MyDrive/CNN_data/test_label.npy')
test_features

,Pictures,Encoding
0,"[[[80, 79, 74], [56, 55, 50], [54, 53, 48], [4...",14
1,"[[[10, 15, 19], [10, 15, 19], [11, 16, 19], [1...",14
2,"[[[232, 231, 247], [250, 249, 255], [235, 234,...",14
3,"[[[97, 75, 77], [98, 76, 78], [99, 77, 79], [1...",14
4,"[[[19, 46, 55], [19, 46, 53], [20, 47, 54], [2...",14
...,...,...
711,"[[[64, 70, 84], [64, 70, 82], [65, 72, 82], [6...",11
712,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",11
713,"[[[127, 178, 241], [124, 176, 236], [124, 176,...",11
714,"[[[71, 97, 122], [70, 91, 118], [66, 86, 113],...",11


In [55]:
import torch
import torch.nn as nn
import numpy as np
import cv2
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F

In [57]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, training_set, is_train = False):
    super().__init__()
    if is_train:
      self.transform = transforms.Compose([transforms.Resize((256,256)), transforms.RandomHorizontalFlip(), transforms.ToTensor()])
    else:
      self.transform = transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()])
    self.dataset = training_set

  def __getitem__(self, index):
    image = self.dataset.iloc[index]['Pictures']
    image = Image.fromarray(image)
    image = self.transform(image)
    label = self.dataset.iloc[index]['Encoding']
    return image, label
  
  def __len__(self):
    return self.dataset.shape[0]

In [58]:
train, val = train_test_split(train_features, train_size=0.8, random_state=41)


In [59]:
train = Dataset(train, is_train = True)
train

In [60]:
val = Dataset(val, is_train = False)
val

In [61]:
test = Dataset(test_features, is_train = False)
test

In [69]:
import torch.utils.data as Data
BATCH_SIZE = 50
train_loader = Data.DataLoader(
    dataset = train,
    batch_size=BATCH_SIZE,
    shuffle=True
)
val_loader = Data.DataLoader(
    dataset = val,
    batch_size = BATCH_SIZE,
    shuffle = False
)
test_loader = Data.DataLoader(
    dataset = test,
    batch_size = 1,
    shuffle = False
)


In [63]:
class NN(nn.Module):
  def __init__(self):
    super(NN,self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = (3,3), stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = (3,3), stride = 1, padding = 1)
    self.pool1 = nn.MaxPool2d(2,2)
    self.pool2 = nn.MaxPool2d(2,2)
    self.pool3 = nn.MaxPool2d(4,4)
    self.fc1 = nn.Linear(32 * 16 * 16, 128)
    self.fc2 = nn.Linear(128, 20)
  
  def forward(self, x):
    x = self.pool1(F.relu(self.conv1(x)))
    x = self.pool2(F.relu(self.conv2(x)))
    x = self.pool3(x)
    x = x.view(x.size(0),-1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    out = x
    return out
  

In [64]:
net = NN()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss_func = torch.nn.CrossEntropyLoss()
model_path = '/content/drive/MyDrive/CNN_data/'
EPOCH = 30

for epoch in range(EPOCH):
  accuracy_train = 0
  for step,(batch_x,batch_y) in enumerate(train_loader):
    pred_y = net(batch_x)
    loss = loss_func(pred_y,batch_y)
    pred_y = torch.max(pred_y, 1)[1].numpy()  
    accuracy_train = accuracy_train + float((pred_y == batch_y.numpy()).astype(int).sum())
    if step % 10 == 0:
      print(f"[TRAINING {epoch}]   loss: {loss:.3f}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  accuracy_train = accuracy_train / (len(train_features) * 0.8)
  print('Epoch: ', epoch, '| train accuracy: %.2f' % accuracy_train)

  accuracy_val = 0
  for step,(batch_x,batch_y) in enumerate(val_loader):
    val_output = net(batch_x)
    pred_y = torch.max(val_output, 1)[1].numpy()  
    accuracy_val = accuracy_val + float((pred_y == batch_y.numpy()).astype(int).sum())
  accuracy_val = accuracy_val / (len(train_features) * 0.2)
  print('Epoch: ', epoch, '| val accuracy: %.2f' % accuracy_val)

  torch.save(net.state_dict(), os.path.join(model_path, str(epoch)+'_val.ckpt'))
 



[TRAINING 0]   loss: 2.993
[TRAINING 0]   loss: 2.942
[TRAINING 0]   loss: 2.738
Epoch:  0 | train accuracy: 0.12
Epoch:  0 | val accuracy: 0.27
[TRAINING 1]   loss: 2.726
[TRAINING 1]   loss: 2.164
[TRAINING 1]   loss: 2.605
Epoch:  1 | train accuracy: 0.29
Epoch:  1 | val accuracy: 0.33
[TRAINING 2]   loss: 2.205
[TRAINING 2]   loss: 2.300
[TRAINING 2]   loss: 2.040
Epoch:  2 | train accuracy: 0.34
Epoch:  2 | val accuracy: 0.34
[TRAINING 3]   loss: 2.131
[TRAINING 3]   loss: 1.660
[TRAINING 3]   loss: 1.930
Epoch:  3 | train accuracy: 0.39
Epoch:  3 | val accuracy: 0.38
[TRAINING 4]   loss: 1.838
[TRAINING 4]   loss: 1.953
[TRAINING 4]   loss: 1.566
Epoch:  4 | train accuracy: 0.42
Epoch:  4 | val accuracy: 0.37
[TRAINING 5]   loss: 1.813
[TRAINING 5]   loss: 1.846
[TRAINING 5]   loss: 1.640
Epoch:  5 | train accuracy: 0.48
Epoch:  5 | val accuracy: 0.43
[TRAINING 6]   loss: 1.681
[TRAINING 6]   loss: 1.612
[TRAINING 6]   loss: 1.574
Epoch:  6 | train accuracy: 0.49
Epoch:  6 | val 

In [71]:
net.load_state_dict(torch.load(os.path.join(model_path, 'best_val.ckpt')))
final_prediction = []
for step,(batch_x,batch_y) in enumerate(test_loader):
  test_output = net(batch_x)
  pred_y = torch.max(test_output, 1)[1].numpy()  
  final_prediction.append(pred_y[0])
final_prediction


[9,
 14,
 13,
 14,
 3,
 8,
 9,
 14,
 14,
 9,
 11,
 9,
 14,
 9,
 14,
 18,
 5,
 14,
 14,
 14,
 19,
 14,
 13,
 14,
 5,
 14,
 14,
 9,
 14,
 5,
 14,
 14,
 18,
 4,
 16,
 11,
 17,
 14,
 14,
 2,
 2,
 4,
 17,
 4,
 15,
 14,
 19,
 19,
 3,
 19,
 17,
 13,
 5,
 19,
 19,
 19,
 7,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 9,
 15,
 14,
 14,
 7,
 19,
 19,
 19,
 19,
 19,
 19,
 10,
 19,
 19,
 5,
 10,
 5,
 14,
 18,
 14,
 7,
 14,
 6,
 7,
 5,
 9,
 14,
 5,
 5,
 17,
 10,
 13,
 13,
 9,
 14,
 5,
 5,
 7,
 8,
 9,
 17,
 5,
 11,
 5,
 17,
 8,
 4,
 8,
 8,
 8,
 5,
 17,
 17,
 17,
 17,
 17,
 17,
 5,
 17,
 17,
 17,
 1,
 8,
 4,
 7,
 17,
 4,
 17,
 14,
 17,
 18,
 10,
 15,
 17,
 17,
 18,
 14,
 4,
 17,
 17,
 17,
 8,
 7,
 4,
 15,
 5,
 17,
 17,
 15,
 7,
 5,
 15,
 1,
 16,
 15,
 15,
 15,
 16,
 9,
 9,
 1,
 15,
 15,
 17,
 5,
 5,
 4,
 17,
 16,
 18,
 16,
 11,
 3,
 16,
 17,
 3,
 3,
 3,
 3,
 3,
 15,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 3,
 2,
 3,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12

In [74]:
pd.DataFrame(final_prediction).to_csv('NN_prediction.csv')